<a href="https://colab.research.google.com/github/nicoelbert/vehicleroutingproblem/blob/main/vehicle_routing_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Vehicle Routing Problem
vehicle routing problem on complete information using simmulated annealing

#Preparation

In [1]:
!git clone 'https://github.com/nicoelbert/vehicleroutingproblem'

fatal: destination path 'vehicleroutingproblem' already exists and is not an empty directory.


In [2]:
#import packages
import pandas as pd

import sys
#import classes


!git pull
%cd vehicleroutingproblem

from classes import classes as cl

fatal: not a git repository (or any of the parent directories): .git
/content/vehicleroutingproblem


In [3]:
!git pull
!ls

Already up to date.
classes        README.md	      vehicle_routing_solver.ipynb
original_data  vehicleroutingproblem


##Data Import

In [12]:
#depot masterdata

#load Masterdata from csv
url = 'https://raw.githubusercontent.com/nicoelbert/vehicleroutingproblem/main/original_data/Depots.csv'
df_depots = pd.read_csv(url)


#load basetable
url = 'https://raw.githubusercontent.com/nicoelbert/vehicleroutingproblem/main/original_data/preparedJobs.csv'
df_jobs_org = pd.read_csv(url)

#create and fill dict_depots

dict_depots = {}

for index, row in df_depots.iterrows():
  depot = cl.Depot(row['Werk'],row['Werk_lon'],row['Werk_lat'])
  dict_depots[row['Werk']] = depot

print("{} depots created".format(len(dict_depots)))


#create and fill plants from df_jobs_org
dict_plants = {}

list_plant_names = df_jobs_org['Werk'].unique() 

for pn in list_plant_names:
  row = df_jobs_org[df_jobs_org['Werk'] == pn].iloc[0]
  plant = cl.Plant(row.Werk,row.Werk_lon,row.Werk_lat)
  dict_plants[plant.name] = plant

print("{} plants created".format(len(dict_plants)))


#create and fill sites from df_jobs_org

dict_sites = {}

df_sites = df_jobs_org[['plz','lon','lat']].copy()
df_sites.drop_duplicates(inplace=True)


for index, row in df_sites.iterrows():
  site = cl.Site(row.plz,row.lon,row.lat)
  dict_sites[site.name] = site


print("{} sites created".format(len(dict_sites)))


#create and fill jobs from df_jobs_org


dict_jobs = {}

for index, row in df_jobs_org.iterrows():
  plant = dict_plants[row.Werk]
  site_name = str(row.plz) + '/' +str(row.lon) + '/' + str(row.lat)
  site = dict_sites[site_name]
  job = cl.Job(row.id, row.key, plant,row.Maschinentechnik,row.Material,row.Start_clean,row.End, row.Werktage, site )
  dict_jobs[job.id] = job



print("{} jobs created".format(len(dict_jobs)))


15 depots created
8 plants created
6166 sites created
53178 jobs created


In [6]:
#next steps:
#-> create scatterplot for depots,plants and sights
#-> evaluate number of jobs for start and end dates


15 depots created
8 plants created
6166 sites created
53178 jobs created


##Backup

In [ ]:
#backup
"""
"Evaluation of Constructionsites"
print(len(df_jobs['plz'].unique()))
print(len(df_jobs['lon'].unique()))
print(len(df_jobs['lat'].unique()))
#print(len(df_jobs[['lon','lat']]()))

df_sites = df_jobs[['lon','lat']].copy()

print(df_sites.shape)

df_sites.drop_duplicates(inplace=True)


print(df_sites.shape)
k.key
"""